# Define Tools

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Tuesday, May 20, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-20', 'content': 'San Francisco, California Weather: Tuesday, May 20, 2025. Cloudy weather, overcast skies with clouds. Day 66°. Night 52°. Precipitation 0 %.', 'score': 0.94339466}, {'title': 'Weather in San Francisco in May 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/may-2025/', 'content': 'Detailed ⚡ San Francisco Weather Forecast for May 2025 – day/night 🌡️ temperatures, precipitations – World-Weather ... Tuesday, 20 May. +52°. Day. +61', 'score': 0.91306627}]


# Usage Language Model (Google Gemini)

In [5]:


from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi I am arafat")])
response.content

"Hi Arafat! It's nice to meet you. How can I help you today?"